<a href="https://colab.research.google.com/github/Sparten-Ashvinee/ERA/blob/main/S18/VAE_cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/U-NET/

/content/drive/MyDrive/U-NET


In [13]:
!ls

 new_pet_data  'UNet from scratch.ipynb'


In [49]:
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torchvision
import numpy as np
import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200

In [50]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [139]:
latent_dims = 2
autoencoder = VariationalAutoencoder(latent_dims).to(device) # GPU

data = torch.utils.data.DataLoader(
        torchvision.datasets.CIFAR10('./cifar10_data',
               transform=torchvision.transforms.ToTensor(),
               download=True),
        batch_size=25,
        shuffle=True)

Files already downloaded and verified


####Assigning random labels

In [140]:
dataset = iter(data)
for i in range(1):
  img, lab = next(dataset)
  idx = torch.randperm(lab.nelement())
  lab = lab.view(-1)[idx].view(lab.size())
  lab = torch.nn.functional.one_hot(lab, num_classes=10)

  dataset_new = torch.utils.data.TensorDataset(img, lab)
  data_loader_ = torch.utils.data.DataLoader(dataset_new, batch_size=25, shuffle=True)


In [141]:
img.shape

torch.Size([25, 3, 32, 32])

In [142]:
class Encoder(nn.Module):
    def __init__(self, latent_dims):
        super(Encoder, self).__init__()
        self.linear1 = nn.Linear(1024, 512)
        self.linear2 = nn.Linear(512, latent_dims)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.linear1(x))
        return self.linear2(x)

In [143]:
class Decoder(nn.Module):
    def __init__(self, latent_dims):
        super(Decoder, self).__init__()
        self.linear1 = nn.Linear(latent_dims, 512)
        self.linear2 = nn.Linear(512, 1024)

    def forward(self, z):
        z = F.relu(self.linear1(z))
        z = torch.sigmoid(self.linear2(z))
        return z.reshape((-1, 1, 32, 32))

In [162]:
class VariationalEncoder(nn.Module):
    def __init__(self, latent_dims):
        super(VariationalEncoder, self).__init__()
        self.linear1 = nn.Linear(1024, 25)
        self.linear2 = nn.Linear(512, latent_dims)
        self.linear3 = nn.Linear(512, latent_dims)

        self.N = torch.distributions.Normal(0, 1)
        self.N.loc = self.N.loc.cuda() # hack to get sampling on the GPU
        self.N.scale = self.N.scale.cuda()
        self.kl = 0

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.linear1(x))
        mu =  self.linear2(x)
        sigma = torch.exp(self.linear3(x))
        z = mu + sigma*self.N.sample(mu.shape)
        self.kl = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()
        return z

In [163]:
class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = VariationalEncoder(latent_dims)
        self.decoder = Decoder(latent_dims)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

In [164]:
def loss_function(x, x_hat, mean, log_var):
    reproduction_loss = nn.functional.binary_cross_entropy(x_hat, x, reduction='sum')
    KLD      = - 0.5 * torch.sum(1+ log_var - mean.pow(2) - log_var.exp())
    return reproduction_loss + KLD

In [165]:
def train(autoencoder, data, epochs=20):
    opt = torch.optim.Adam(autoencoder.parameters())
    for epoch in range(epochs):
        for x, y in data:
            x = x.to(device) # GPU
            opt.zero_grad()
            x_hat = autoencoder(x)
            loss = ((x - x_hat)**2).sum()
            #loss = torch.nn.KLDivLoss(size_average=False)(F.log_softmax(x, -1), x_hat)
            print('Training loss for epoch: ', epoch)
            print('Loss: ', loss)
            loss.backward()
            opt.step()
    return autoencoder

In [166]:
vae = VariationalAutoencoder(latent_dims).to(device) # GPU

In [ ]:
autoencoder = train(vae, data_loader_, epochs=10)

In [169]:
def plot_reconstructed(autoencoder, r0=(-5, 10), r1=(-10, 5), n=5):
    w = 32
    img = np.zeros((n*w, n*w))
    for i, y in enumerate(np.linspace(*r1, n)):
        for j, x in enumerate(np.linspace(*r0, n)):
            z = torch.Tensor([[x, y]]).to(device)
            x_hat = autoencoder.decoder(z)
            x_hat = x_hat.reshape(28, 28).to('cpu').detach().numpy()
            img[(n-1-i)*w:(n-1-i+1)*w, j*w:(j+1)*w] = x_hat
    plt.imshow(img, extent=[*r0, *r1])

In [ ]:
plot_reconstructed(vae, r0=(-3, 3), r1=(-3, 3))